In [25]:
from main import *

class apiHandler:
    # Configuration
    endpoint = '/agents?select=lastKeepAlive&select=id&status=active'
    protocol = 'https'
    host = '54.159.199.49'
    port = '55000'
    user = 'wazuh-wui'
    password = 'uvVZM6eL1tb.1VELhQ1SxUo7RxUauw+N'

    # Variables
    base_url = f"{protocol}://{host}:{port}"
    login_url = f"{base_url}/security/user/authenticate"
    basic_auth = f"{user}:{password}".encode()
    headers = {}
    allAgents = None
    # Disable insecure https warnings (for self-signed SSL certificates)
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # Functions
    def get_response(self, url, headers, verify=False):
        request_result = requests.get(url, headers=headers, verify=verify)
        if request_result.status_code == 200:
            return json.loads(request_result.content.decode())
        else:
            raise Exception(f"Error obtaining response: {request_result.json()}")
            
    def get_token(self):
        self.headers = {'Authorization': f'Basic {b64encode(self.basic_auth).decode()}'}
        self.headers['Authorization'] = f'Bearer {self.get_response(self.login_url, self.headers)["data"]["token"]}'

    def get_agents(self):
        url = self.base_url + '/agents'
        self.allAgents = self.get_response(url, self.headers)
        return self.allAgents
    
    def get_vul_by_crit(self, severity):
        total_vulnerabilities = 0
        url_start = "/vulnerability/"
        url_end = "/summary/severity"
        url_mid = ""
        
        for i in range(1, jsonApi["data"]["total_affected_items"]):
            url_mid = jsonApi["data"]["affected_items"][i]["id"]
            url = self.base_url + url_start + url_mid + url_end
            
            num_crit = self.get_response(url, self.headers)['data']['severity'].get(severity)
            if( num_crit is not None):
                total_vulnerabilities += num_crit
        return total_vulnerabilities
    
    def get_vul_by_key(self, keyWord, limite = 10):
        url_start = "/vulnerability/"
        url_end = f"?pretty=true&search={keyWord}&select=name,condition,status,severity&limit={limite}"
        url_mid = ""
        
        for i in range(1, jsonApi["data"]["total_affected_items"]):
            url_mid = jsonApi["data"]["affected_items"][i]["id"]
            url = self.base_url + url_start + url_mid + url_end
            
            response = self.get_response(url, self.headers)['data']
            if(url):print(url)
            if(response):print(response)
        
            

apiTest = apiHandler()

''' Generar tokens '''
apiTest.get_token()
#print(apiTest.headers["Authorization"])

''' Sacar la info de todos los agentes '''
jsonApi = apiTest.get_agents()
#print(jsonApi["data"].keys())
#print(jsonApi["data"]["total_affected_items"])
#print(jsonApi["data"]["affected_items"][11])

''' Sacar el numero total de vulnerabilidades por severidad '''
#print("HIGH:", apiTest.get_vul_by_crit("High"))

''' Sacar vulnerabilidades por palabra clave con limite default de 10 por agente '''
print("Palabra: ",apiTest.get_vul_by_key("Adobe"))

ConnectionError: HTTPSConnectionPool(host='54.159.199.49', port=55000): Max retries exceeded with url: /security/user/authenticate (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000270F3554EB0>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))

In [1]:
# IMPRIMIR severidad de Vulnerabilidades
from wazuhApi import *

''' Sacar el numero total de vulnerabilidades por severidad '''
rangos = ["High","Medium","Critical","Low"]
for i in rangos:
    print(f"Level: {i} -> ", json.dumps(apiTest.get_vul_by_crit(i), indent=4, sort_keys=True))

''' Sacar vulnerabilidades por palabra clave con limite default de 10 por agente '''
#print("Palabra: ",apiTest.get_vul_by_key("Adobe"))
#print(json.dumps(apiTest.get_vul_by_key("Adobe"), indent=4, sort_keys=True))#

HIGH: 1616
https://54.145.241.208:55000/vulnerability/001?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/002?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/003?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/004?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/005?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_ite

' Sacar vulnerabilidades por palabra clave con limite default de 10 por agente '

In [23]:
# Librerias

#from wazuhApi import *
from main import *
from base64 import b64encode
import json
import requests  
import urllib3
import os
import tkinter as tk
from tkinter import ttk
import tkinter.font as tkFont
from tkinter import *
import pprint
from tkinter.ttk import *
import json

In [27]:
# Creamos la ventana y titulo de la ventana
ventana = tk.Tk()
ventana.iconbitmap('assets/wazuh.ico')
ventana.title("Los datos importantes")
ventana.configure(bg="skyblue")
menuBar(ventana)

# Dimensiones y posicion de inicio de la pantalla.
ventana.config(width=500, height=500)
altura = ventana.winfo_reqheight()
anchura = ventana.winfo_reqwidth()
altura_pantalla = ventana.winfo_screenheight()
anchura_pantalla = ventana.winfo_screenwidth()
ventana.geometry(
    f'{altura}x{anchura}+{int(anchura_pantalla/2-(anchura/2))}+{int(altura_pantalla/2-(altura/2))}')
ventana.resizable(False, False)

style=ttk.Style()
style.theme_use('alt')
style.configure("TScrollbar", background="blue")

widthListBox = anchura*.80
heightListBox = altura*.80
anchoSBY = anchura
altoSBX = altura

# Estilos y tamaños de letra
fontStyle_Labels = tkFont.Font(family="Lucida Grande", size=15)
fontStyle_Text = tkFont.Font(family="Helvetica bold", size=18)

Label(ventana, text="Hello World!", font=fontStyle_Text,anchor="center", background="#237235").pack(pady=10)

# Estilo para el texto
rangos = ["High","Medium","Critical","Low"]
# for i in rangos:
#     #print(f"Level: {i} -> ", json.dumps(apiTest.get_vul_by_crit(i), indent=4, sort_keys=True))
#     JP = (json.dumps(apiTest.get_vul_by_crit(i), indent=4, sort_keys=True))
#     # print(JP)  # JP es un String
#     datos_diccionario = json.loads(JP)

listbox = tk.Listbox(font=fontStyle_Text)
# value = datos_diccionario["data"]["affected_items"]
# listbox.insert(
#     tk.END, *(
#         f'ID: {value[i]["id"]}, \n Last Pick: {value[i]["lastKeepAlive"]}' 
#         for i in range(len(value))
#     )
# )

# LISTBOX positions and funtion about scrollbar
listbox.place(x=int(anchura*.9)-widthListBox, y=int(altura*.1), width=widthListBox, height=heightListBox)
scrollbar = ttk.Scrollbar(orient=tk.VERTICAL, command=listbox.yview)
scrollbar2 = ttk.Scrollbar(orient=tk.HORIZONTAL, command=listbox.xview)
listbox.config(yscrollcommand=scrollbar.set)
listbox.config(xscrollcommand=scrollbar2.set)
high(listbox)

# SCROLLBAR dimentions
scrollbar.place(x=anchoSBY-20, y=15, height=altura-40)
scrollbar2.place(x=5, y=altoSBX-20, width=anchoSBY-25)

ventana.mainloop()


good bye :)


{
    "data": {
        "affected_items": [
            {
                "dateAdd": "2023-02-27T23:40:01Z",
                "id": "000",
                "ip": "127.0.0.1",
                "lastKeepAlive": "9999-12-31T23:59:59Z",
                "manager": "ip-172-31-32-91",
                "name": "ip-172-31-32-91",
                "node_name": "node01",
                "os": {
                    "arch": "x86_64",
                    "codename": "Jammy Jellyfish",
                    "major": "22",
                    "minor": "04",
                    "name": "Ubuntu",
                    "platform": "ubuntu",
                    "uname": "Linux |ip-172-31-32-91 |5.15.0-1028-aws |#32-Ubuntu SMP Mon Jan 9 12:28:07 UTC 2023 |x86_64",
                    "version": "22.04.1 LTS"
                },
                "registerIP": "127.0.0.1",
                "status": "active",
                "version": "Wazuh v4.3.10"
            },
            {
                "configSum": "ab73af4169